# WaveBot MASK Workshop

In [1]:
import autograd.numpy as np
import capytaine as cpy
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import wecopttool as wot


## 1. WEC object
### Mesh

We already include the mesh for the WaveBot in WecOptTool. For other devices you would have to create the geometry and mesh. See the WaveBot mesh as an example.

In [2]:
wb = wot.geom.WaveBot()
mesh_size_factor = 0.5 # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)


### DOFs
We will model a 1 DOF (heave) case.

In [3]:
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")
fb.add_translation_dof(name="Heave")
ndof = fb.nb_dofs


### Hydrostatics
We will use WecOptTool's `hydrostatics` module to get the mass and hydrostatic stiffness. 

In [4]:
hydro_stiffness = wot.hydrostatics.stiffness_matrix(fb).values  # N/m
mass = wot.hydrostatics.inertia_matrix(fb).values  # kg


### Frequencies
Choose frequencies based on your waves & whether you include non-linearities. (Discussion) 

In [5]:
f1 = 0.05  # Hz
nfreq = 50
freq = wot.frequency(f1, nfreq, False)  # Hz, # False -> no zero frequency


### BEM
We provide a function that runs Capytaine. (Take a look)

In [6]:
bem_data = wot.run_bem(fb, freq)


The resolution of the mesh 'WaveBot' of the body 'WaveBot_immersed' might be insufficient for the wavelength λ=7.97e-01.
This warning appears because the largest panel of this mesh has radius 1.01e-01 > λ/8.
The resolution of the mesh 'WaveBot' of the body 'WaveBot_immersed' might be insufficient for the wavelength λ=7.43e-01.
This warning appears because the largest panel of this mesh has radius 1.01e-01 > λ/8.
The resolution of the mesh 'WaveBot' of the body 'WaveBot_immersed' might be insufficient for the wavelength λ=6.94e-01.
This warning appears because the largest panel of this mesh has radius 1.01e-01 > λ/8.
The resolution of the mesh 'WaveBot' of the body 'WaveBot_immersed' might be insufficient for the wavelength λ=6.50e-01.
This warning appears because the largest panel of this mesh has radius 1.01e-01 > λ/8.
The resolution of the mesh 'WaveBot' of the body 'WaveBot_immersed' might be insufficient for the wavelength λ=6.10e-01.
This warning appears because the largest panel 

### PTO

We will define our PTO based on a PTO impedance (refer to slides and papers). The PTO will be used to 
1. add an additional force to our dynamic equations
1. the objective function: average electrical power

In [7]:
## PTO impedance definition
omega = bem_data.omega.values  # rad/s
gear_ratio = 12.0  # rad/m
torque_constant = 6.7  # N*m/A
winding_resistance = 0.5  # Ω
winding_inductance = 0.0  # H
drivetrain_inertia = 2.0  # kg*m^2
drivetrain_friction = 1.0  # N*m*s/rad
drivetrain_stiffness = 0.0  # N*m/rad

drivetrain_impedance = (1j*omega*drivetrain_inertia +
                        drivetrain_friction +
                        1/(1j*omega)*drivetrain_stiffness)

winding_impedance = winding_resistance + 1j*omega*winding_inductance


pto_impedance_11 = -1* gear_ratio**2 * drivetrain_impedance
off_diag = np.sqrt(3.0/2.0) * torque_constant * gear_ratio
pto_impedance_12 = -1*(off_diag+0j) * np.ones(omega.shape)
pto_impedance_21 = -1*(off_diag+0j) * np.ones(omega.shape)
pto_impedance_22 = winding_impedance
pto_impedance = np.array([[pto_impedance_11, pto_impedance_12],
                          [pto_impedance_21, pto_impedance_22]])


In [8]:
f_max = 1000

def saturated_pi(pto, wec, x_wec, x_opt, waves=None, nsubsteps=1):
    force_td_tmp = wot.pto.controller_pi(pto, wec, x_wec, x_opt, waves, nsubsteps)
    force_td = np.clip(force_td_tmp, -1*f_max, f_max)
    return force_td


In [9]:
## Create PTO object
pto_name = ['PTO_Heave',]
kinematics = np.eye(ndof)
controller = saturated_pi
loss = None
pto = wot.pto.PTO(ndof, kinematics, controller, pto_impedance, loss, pto_name)


### Constraints

In [10]:
constraints = [
    ]


### Additional forces

In [11]:
f_add = {
    'PTO': pto.force_on_wec,
}


### WEC Object

In [12]:
wec = wot.WEC.from_bem(bem_data,
                       inertia_matrix=mass,
                       hydrostatic_stiffness=hydro_stiffness,
                       constraints=constraints,
                       friction=None,
                       f_add=f_add,
)


## 2. Waves

In [13]:
regular = True

if regular:
    amplitude = 0.2
    wavefreq = 0.3
    waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude)
else:
    fp = 0.3
    Hm0 = 0.2
    omnidirectional_spectrum = wot.waves.omnidirectional_spectrum(
        f1=f1,
        nfreq=nfreq,
        spectrum_func=lambda f: wot.waves.jonswap_spectrum(freq=f, fp=fp, hs=Hm0, gamma=3.3),
        spectrum_name="JONSWAP",
    )
    waves = wot.waves.long_crested_wave(omnidirectional_spectrum, 0, seed=13)


## 3. Objective Function

In [14]:
obj_fun = pto.average_power


## 4. Solve

In [15]:
nstate_opt = 2


In [17]:
# Solve
scale_x_wec = 1e3 # 1e1
scale_x_opt = 1 # 1e-3
scale_obj = 1 # 1e-2

results = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    scale_x_wec=scale_x_wec,
    scale_x_opt=scale_x_opt,
    scale_obj=scale_obj,
)

print(f'Optimal average electrical power: {results.fun} W')

# Post-process
nsubstep_postprocess = 4
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubstep_postprocess)
pto_fdom, pto_tdom = pto.post_process(wec, results, waves, nsubstep_postprocess)


Optimization terminated successfully    (Exit mode 0)
            Current function value: -172.7631699907384
            Iterations: 76
            Function evaluations: 113
            Gradient evaluations: 76
Optimal average electrical power: -172.7631699907384 W


## 5. Solve PI

In [18]:
controller_pi = wot.pto.controller_pi
pto_pi = wot.pto.PTO(ndof, kinematics, controller_pi, pto_impedance, loss, pto_name)
f_add_pi = {'PTO': pto_pi.force_on_wec,}
wec_pi = wot.WEC.from_bem(bem_data,
                       inertia_matrix=mass,
                       hydrostatic_stiffness=hydro_stiffness,
                       constraints=constraints,
                       friction=None,
                       f_add=f_add_pi,
)
obj_fun_pi = pto_pi.average_power

scale_x_wec_pi = 1e1
scale_x_opt_pi = 1e-3
scale_obj_pi = 1e-2

results_pi = wec_pi.solve(
    waves,
    obj_fun_pi,
    nstate_opt,
    scale_x_wec=scale_x_wec_pi,
    scale_x_opt=scale_x_opt_pi,
    scale_obj=scale_obj_pi,
)

print(f'Optimal average electrical power: {results_pi.fun} W')

# Post-process
# nsubstep_postprocess = 4
wec_fdom_pi, wec_tdom_pi = wec_pi.post_process(results_pi, waves, nsubstep_postprocess)
pto_fdom_pi, pto_tdom_pi = pto_pi.post_process(wec_pi, results_pi, waves, nsubstep_postprocess)


Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.958934829970642
            Iterations: 35
            Function evaluations: 40
            Gradient evaluations: 35
Optimal average electrical power: -295.8934829970642 W


## 6. Unstructured, constrained

In [ ]:
nstate_opt_uc = 2*nfreq

controller_uc = None
pto_uc = wot.pto.PTO(ndof, kinematics, controller_uc, pto_impedance, loss, pto_name)
f_add_uc = {'PTO': pto_uc.force_on_wec,}


def const_f_pto(wec, x_wec, x_opt, waves):
    f = pto.force_on_wec(wec, x_wec, x_opt, waves, nsubsteps)
    return f_max - np.abs(f.flatten())

constraints_uc = [
    {'type': 'ineq', 'fun': const_f_pto,},
]

wec_uc = wot.WEC.from_bem(bem_data,
                       inertia_matrix=mass,
                       hydrostatic_stiffness=hydro_stiffness,
                       constraints=constraints_uc,
                       friction=None,
                       f_add=f_add_uc,
)
obj_fun_uc = pto_uc.average_power

scale_x_wec_uc = 1e1
scale_x_opt_uc = 1e-3
scale_obj_uc = 1e-2

results_uc = wec_uc.solve(
    waves,
    obj_fun_uc,
    nstate_opt_uc,
    scale_x_wec=scale_x_wec_uc,
    scale_x_opt=scale_x_opt_uc,
    scale_obj=scale_obj_uc,
)

print(f'Optimal average electrical power: {results_uc.fun} W')

# Post-process
# nsubstep_postprocess = 4
wec_fdom_uc, wec_tdom_uc = wec_uc.post_process(results_uc, waves, nsubstep_postprocess)
pto_fdom_uc, pto_tdom_uc = pto_uc.post_process(wec_uc, results_uc, waves, nsubstep_postprocess)


## 7.Plots

In [ ]:
n = 4
fig, ax = plt.subplots(n,1, sharex=True, figsize=(10,10))
plt.xlim([0, wec.tf])
cmap = plt.get_cmap("tab10")

# excitation force
i = 0
y = np.squeeze(wec_tdom['force'].sel(type='Froude_Krylov') +
               wec_tdom['force'].sel(type='diffraction'))
y_pi = np.squeeze(wec_tdom_pi['force'].sel(type='Froude_Krylov') +
                  wec_tdom_pi['force'].sel(type='diffraction'))

ax[i].plot(0, y_pi[0], 'o', color=cmap(1), label='_nolegend_')
ax[i].plot(wec_pi.tf, y_pi[0], 'o', color=cmap(1), markerfacecolor='none', label='_nolegend_')
ax[i].plot([wec_pi.time_nsubsteps(nsubstep_postprocess)[-1], wec_pi.tf], [y_pi[-1], y_pi[0]], '-', color=cmap(1), label='_nolegend_')
y_pi.plot(color=cmap(1), ax=ax[i], label='PI'); ax[i].set_xlabel(""); ax[i].set_title("")
# ax[i].plot(wec_pi.)

ax[i].plot(0, y[0], 'o', color=cmap(0), label='_nolegend_')
ax[i].plot(wec.tf, y[0], 'o', color=cmap(0), markerfacecolor='none', label='_nolegend_')
ax[i].plot([wec.time_nsubsteps(nsubstep_postprocess)[-1], wec.tf], [y[-1], y[0]], '--', color=cmap(0), label='_nolegend_')

y.plot(color=cmap(0), linestyle='--', ax=ax[i], label='Sat. PI'); ax[i].set_xlabel(""); ax[i].set_title("")

ax[i].set_ylabel('excitation force [N]')

# PTO force
i = 1
y = np.squeeze(pto_tdom['force'])
y_pi = np.squeeze(pto_tdom_pi['force'])

ax[i].plot([0, wec.tf], [f_max, f_max], 'k--', linewidth=0.5, label='_nolegend_')
ax[i].plot([0, wec.tf], [-f_max, -f_max], 'k--', linewidth=0.5, label='_nolegend_')
ax[i].plot([0, wec.tf], [0, 0], 'k-', linewidth=0.5, label='_nolegend_')

ax[i].plot(0, y_pi[0], 'o', color=cmap(1), label='_nolegend_')
ax[i].plot(wec_pi.tf, y_pi[0], 'o', color=cmap(1), markerfacecolor='none', label='_nolegend_')
ax[i].plot([wec_pi.time_nsubsteps(nsubstep_postprocess)[-1], wec_pi.tf], [y_pi[-1], y_pi[0]], '-', color=cmap(1), label='_nolegend_')
y_pi.plot(color=cmap(1), ax=ax[i], label='PI'); ax[i].set_xlabel(""); ax[i].set_title("")

ax[i].plot(0, y[0], 'o', color=cmap(0), label='_nolegend_')
ax[i].plot(wec.tf, y[0], 'o', color=cmap(0), markerfacecolor='none', label='_nolegend_')
ax[i].plot([wec.time_nsubsteps(nsubstep_postprocess)[-1], wec.tf], [y[-1], y[0]], '--', color=cmap(0), label='_nolegend_')

y.plot(color=cmap(0), linestyle='--', ax=ax[i], label='Sat. PI') ; ax[i].set_xlabel(""); ax[i].set_title("")

ax[i].set_ylabel('PTO force [N]')

# position
i = 2
y = np.squeeze(wec_tdom['pos'])

y_pi = np.squeeze(wec_tdom_pi['pos'])
ax[i].plot(0, y_pi[0], 'o', color=cmap(1), label='_nolegend_')
ax[i].plot(wec_pi.tf, y_pi[0], 'o', color=cmap(1), markerfacecolor='none', label='_nolegend_')
ax[i].plot([wec_pi.time_nsubsteps(nsubstep_postprocess)[-1], wec_pi.tf], [y_pi[-1], y_pi[0]], '-', color=cmap(1), label='_nolegend_')
y_pi.plot(color=cmap(1), ax=ax[i], label='PI'); ax[i].set_xlabel(""); ax[i].set_title("")

ax[i].plot(0, y[0], 'o', color=cmap(0), label='_nolegend_')
ax[i].plot(wec.tf, y[0], 'o', color=cmap(0), markerfacecolor='none', label='_nolegend_')
ax[i].plot([wec.time_nsubsteps(nsubstep_postprocess)[-1], wec.tf], [y[-1], y[0]], '--', color=cmap(0), label='_nolegend_')

y.plot(color=cmap(0), linestyle='--', ax=ax[i], label='Sat. PI'); ax[i].set_xlabel(""); ax[i].set_title("")

ax[i].set_ylabel('position [m]')

# velocity
i = 3
y = np.squeeze(wec_tdom['vel'])

y_pi = np.squeeze(wec_tdom_pi['vel'])
ax[i].plot(0, y_pi[0], 'o', color=cmap(1), label='_nolegend_')
ax[i].plot(wec_pi.tf, y_pi[0], 'o', color=cmap(1), markerfacecolor='none', label='_nolegend_')
ax[i].plot([wec_pi.time_nsubsteps(nsubstep_postprocess)[-1], wec_pi.tf], [y_pi[-1], y_pi[0]], '-', color=cmap(1), label='_nolegend_')
y_pi.plot(color=cmap(1), ax=ax[i], label='PI'); ax[i].set_xlabel(""); ax[i].set_title("")

ax[i].plot(0, y[0], 'o', color=cmap(0), label='_nolegend_')
ax[i].plot(wec.tf, y[0], 'o', color=cmap(0), markerfacecolor='none', label='_nolegend_')
ax[i].plot([wec.time_nsubsteps(nsubstep_postprocess)[-1], wec.tf], [y[-1], y[0]], '--', color=cmap(0), label='_nolegend_')

y.plot(color=cmap(0), linestyle='--', ax=ax[i], label='Sat. PI'); ax[i].set_xlabel(""); ax[i].set_title("")

ax[i].set_ylabel('velocity [m/s]')

#
plt.xlabel("time [s]")
ax[0].legend()


In [ ]:
print(f"PI gains: {results_pi.x[-2:]}")
print(f"Saturated PI gains: {results.x[-2:]}")


In [ ]:
if not regular:
    wavefreq = fp

plt.figure()
maxfreq = pto_fdom["freq"][-1]/wavefreq
plt.plot([0, maxfreq], [0, 0], 'k-', linewidth=0.5)
plt.plot(pto_fdom_pi["freq"]/wavefreq, np.abs(pto_fdom_pi.force), marker="o", linestyle="None", color=cmap(1))
plt.plot(pto_fdom["freq"]/wavefreq, np.abs(pto_fdom.force), marker="o", linestyle="None", color=cmap(0), alpha=0.5)
plt.title("")
plt.xlabel("frequency ω/ω₀")
plt.ylabel("PTO force [N]")
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.xlim([0, maxfreq])
